In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration
)
from azure.identity import AzureCliCredential, DefaultAzureCredential

In [2]:
credential = AzureCliCredential()

In [ ]:
# enter details of your Azure Machine Learning workspace
subscription_id = ""
resource_group = "soulfriendly"
workspace = "soulfriend"

In [4]:
# get a handle to the workspace
ml_client = MLClient(
    credential, subscription_id, resource_group, workspace
)

In [5]:
# method 1: define an endpoint name
endpoint_name = "mood-classifier-endpoint"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="this is a sample endpoint",
    auth_mode="key"
)

In [31]:
model = Model(path="model.pth")
env = Environment(
    conda_file="conda.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code=".", scoring_script="score.py"
    ),
    instance_type="Standard_D2as_v4",
    instance_count=1,
)

In [32]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Creating local endpoint (mood-classifier-endpoint) .

Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'mood-classifier-endpoint', 'description': 'this is a sample endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('C:/Users/thori/.azureml/inferencing/mood-classifier-endpoint'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x00000199A14A63D0>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [33]:
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Creating local deployment (mood-classifier-endpoint / blue) .
Building Docker image from Dockerfile.
Step 1/6 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest
 ---> 83631454f85b
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> 5c88b8e11f07
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Using cache
 ---> e3086b429727
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> Using cache
 ---> d3b92771a080
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Using cache
 ---> dbdf73e8a552
Step 6/6 : CMD ["conda", "run", "--no-capture-output", "-n", "inf-conda-env", "runsvdir", "/var/runit"]
 ---> Using cache
 ---> 6ec19244ef19
Successfully built 6ec19244ef19
Successfully tagged mood-classifier-endpoint:blue

Starting up endpoint...Done (0m 25s)


ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'mood-classifier-endpoint', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('d:/Thoriq/Project/amls-pytorch-vscode/deployment'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x00000199A22E30D0>, 'model': Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'ac085473d82899c3d3f7d5c5acad2c23', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('d:/Thoriq/Project/amls-pytorch-vscode/deployment'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001

In [35]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32779/score', 'openapi_uri': None, 'name': 'mood-classifier-endpoint', 'description': 'this is a sample endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('d:/Thoriq/Project/amls-pytorch-vscode/deployment'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x00000199A247CB10>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [36]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, local=True, lines=1000
)

'2025-06-01T16:54:20,210969364+00:00 - nginx/run \r\n2025-06-01T16:54:20,210758596+00:00 - rsyslog/run \r\n2025-06-01T16:54:20,216334008+00:00 - gunicorn/run \r\n2025-06-01T16:54:20,219519282+00:00 | gunicorn/run | \r\n2025-06-01T16:54:20,221476136+00:00 | gunicorn/run | ###############################################\r\n2025-06-01T16:54:20,224295579+00:00 | gunicorn/run | AzureML Container Runtime Information\r\n2025-06-01T16:54:20,227306961+00:00 | gunicorn/run | ###############################################\r\n2025-06-01T16:54:20,230760467+00:00 | gunicorn/run | \r\n2025-06-01T16:54:20,834104429+00:00 | gunicorn/run | \r\n2025-06-01T16:54:20,837068135+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20250224.v1\r\n2025-06-01T16:54:20,839013717+00:00 | gunicorn/run | \r\n2025-06-01T16:54:20,841775174+00:00 | gunicorn/run | \r\n2025-06-01T16:54:20,844172257+00:00 | gunicorn/run | PATH environment variable: /opt/miniconda/envs/inf-conda

In [37]:
ml_client.online_endpoints.begin_delete(name=endpoint_name, local=True)

In [6]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

In [7]:
import os
os.listdir("./model/")

['model.pth']

In [8]:
file_model = Model(
    path="./model/model.pth",
    type=AssetTypes.CUSTOM_MODEL,
    name="mood-model",
    description="A model for mood classification",
    tags={"framework": "pytorch", "version": "1.0"},
)
ml_client.models.create_or_update(file_model)

Uploading model.pth (< 1 MB): 239MB [01:19, 2.99MB/s]                            




Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'mood-model', 'description': 'A model for mood classification', 'tags': {'framework': 'pytorch', 'version': '1.0'}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/05e5ad0d-e911-4914-a2b2-7bd73b14d663/resourceGroups/soulfriendly/providers/Microsoft.MachineLearningServices/workspaces/soulfriend/models/mood-model/versions/1', 'Resource__source_path': '', 'base_path': 'd:\\Thoriq\\Project\\amls-pytorch-vscode\\deployment', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x00000170BF469A50>, 'serialize': <msrest.serialization.Serializer object at 0x00000170C59B2ED0>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/05e5ad0d-e911-4914-a2b2-7bd73b14d663/resourceGroups/soulfriendly/workspaces/soulfriend/datastores/workspaceblobstore/paths/LocalUpload/f92d

In [9]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="./conda.yml",
    name="my-env",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'my-env', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/05e5ad0d-e911-4914-a2b2-7bd73b14d663/resourceGroups/soulfriendly/providers/Microsoft.MachineLearningServices/workspaces/soulfriend/environments/my-env/versions/1', 'Resource__source_path': '', 'base_path': 'd:\\Thoriq\\Project\\amls-pytorch-vscode\\deployment', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x00000170C546C910>, 'serialize': <msrest.serialization.Serializer object at 0x00000170C5E74510>, 'version': '1', 'conda_file': {'channels': ['conda-forge'], 'dependencies': ['python=3.8', {'pip': ['azureml-de

In [5]:
model = "azureml:mood-model:1"
env = "azureml:my-env:1"
endpoint_name = "moodclassifier"
 
blue_deployment_with_registered_assets = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code=".", scoring_script="score.py"
    ),
    instance_type="Standard_D2as_v4",
    instance_count=1,
)

In [7]:
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="this is a sample endpoint",
    auth_mode="key"
)

In [8]:
ml_client.online_endpoints.begin_create_or_update(endpoint)

In [9]:
ml_client.online_deployments.begin_create_or_update(blue_deployment_with_registered_assets)

Check: endpoint moodclassifier exists
Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy 'D:\Thoriq\Project\amls-pytorch-vscode\deployment' 'https://soulfriend9406066378.blob.core.windows.net/af650aaf-56f3-4a6b-a88a-47d21639cab4-1l19kpqacy49eq151eof6zp6o2/deployment' 

See https://learn.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading deployment (1826.31 MBs):   6%|6         | 114507917/1826307621 [00:33<04:55, 5798265.81it/s]Queue is full, likely spans will be dropped.
Queue is full, likely spans will be dropped.
Uploading deployment (1826.31 MBs): 100%|##########| 1826307621/1826307621 [09:01<00:00, 3372585.46it/s] 




.........

In [12]:
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint)

HttpResponseError: (BadRequest) The request is invalid.
Code: BadRequest
Message: The request is invalid.
Exception Details:	(InferencingClientCallFailed) {"error":{"code":"Validation","message":"{\"errors\":{\"DeploymentWeights\":[\"Deployments given positive traffic values should be either in a successful or failed state. Unmatched deployments: [blue]\"]},\"type\":\"https://tools.ietf.org/html/rfc9110#section-15.5.1\",\"title\":\"One or more validation errors occurred.\",\"status\":400,\"traceId\":\"00-ccc632996124eec88ec4c3abc65353cc-659738e89f4e82c8-01\"}"}}
	Code: InferencingClientCallFailed
	Message: {"error":{"code":"Validation","message":"{\"errors\":{\"DeploymentWeights\":[\"Deployments given positive traffic values should be either in a successful or failed state. Unmatched deployments: [blue]\"]},\"type\":\"https://tools.ietf.org/html/rfc9110#section-15.5.1\",\"title\":\"One or more validation errors occurred.\",\"status\":400,\"traceId\":\"00-ccc632996124eec88ec4c3abc65353c

In [13]:
ml_client.online_endpoints.get(name=endpoint_name)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://moodclassifier.eastasia.inference.ml.azure.com/score', 'openapi_uri': 'https://moodclassifier.eastasia.inference.ml.azure.com/swagger.json', 'name': 'moodclassifier', 'description': 'Endpoint for mood classification', 'tags': {}, 'properties': {'createdBy': 'abhinaya rajendra', 'createdAt': '2025-06-02T04:25:37.351837+0000', 'lastModifiedAt': '2025-06-02T04:25:37.351837+0000', 'azureml.onlineendpointid': '/subscriptions/05e5ad0d-e911-4914-a2b2-7bd73b14d663/resourcegroups/soulfriendly/providers/microsoft.machinelearningservices/workspaces/soulfriend/onlineendpoints/moodclassifier', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/05e5ad0d-e911-4914-a2b2-7bd73b14d663/providers/Microsoft.MachineLearningServices/locations/eastasia/mfeOperationsStatus/oeidp:af650aaf-56f3-4a6b-a88a-47d21639cab4:7f522150-2188-4cf5-a897-5a7fbbd5f35f?api-version=2022-02-01-p

In [14]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, lines=50
)

ResourceNotFoundError: (ResourceNotFound) The Resource 'Microsoft.MachineLearningServices/workspaces/soulfriend/onlineEndpoints/moodclassifier/deployments/blue' under resource group 'soulfriendly' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.MachineLearningServices/workspaces/soulfriend/onlineEndpoints/moodclassifier/deployments/blue' under resource group 'soulfriendly' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix